# **INSTITUTO DE EDUCAÇÃO SUPERIOR DE BRASÍLIA - IESB**

**TRABALHO FINAL DA MATÉRIA: ** Data Mining e Machine Learning II



* **Professor: ** Marcos
* **Campus: ** Asa Norte
* **Aluna: ** Laís A. de Souza

# **Introdução**

O presente trabalho tem o foco principal de implementar o modelo RandomForest para o conjunto de dados de Home Equity (HMEQ) e analisar se o modelo é capaz de definir se é um bom pagador ou não.
Será utilizado a biblioteca **scikit-learn** *Random Forest Classifier*



In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sn
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
#Ler o arquivo e gravar no DataFrame
df = pd.read_csv('/kaggle/input/hmeq-data/hmeq.csv')

In [ ]:
#Apresentar o cabeçalho e os 5 primeiros registros
df.head()

# Dicionário de dados

* BAD: 1 = cliente inadimplente no empréstimo 0 = empréstimo reembolsado
* LOAN: Montante do pedido de empréstimo
* MORTDUE: Valor devido da hipoteca existente
* VALUE: Valor da propriedade atual
* REASON: DebtCon = consolidação da dívida HomeImp = melhoria da casa
* JOBS: Seis categorias ocupacionais
* YOJ: Anos no emprego atual
* DEROG: Número de principais relatórios depreciativos
* DELINQ: Número de linhas de crédito inadimplentes
* CLAGE: Idade da linha comercial mais antiga em meses
* NINQ: Número de linhas de crédito recentes
* CLNO: Número de linhas de crédito
* DEBTINC: Rácio dívida / rendimento

# Transformação de Dados

Essa etapa é executada para transformar os dados originais em formatos mais apropriados e adequados para a aplicação do modelo:
1. Trasnformar os dados object
1. Retirar os registros missing : https://sigmoidal.ai/como-tratar-dados-ausentes-com-pandas/


In [ ]:
# Verificar quais são as colunas objetct
df.select_dtypes('object').head()

In [ ]:
# Analisando os registros  da coluna REASON
df['REASON'].value_counts()

In [ ]:
# Analisando os registros com os valores missing
pd.isnull(df).sum()

In [ ]:
#Apresentar a quantidade de registros e colunas
df.shape

In [ ]:
#Deletar os registros com dados nulos
df_all = (df.dropna(axis=0))

In [ ]:
#Verificar se ainda existe registros nulos
pd.isnull(df_all).sum()

In [ ]:
#Apresentar a quantidade de registros e colunas após a exclusão dos registros nulos
df_all.shape

In [ ]:
# Transformar os registros 'DebtCon' em 1 e 'HomeImp' em 0
# Na coluna REASON
mapeamento = { 'DebtCon': 1, 'HomeImp': 0}

df_all['REASON'] = df_all['REASON'].replace(mapeamento).astype(int)


In [ ]:
#Verificar a transformação da coluna 'REASON'
df_all.head()

In [ ]:
# Analisando os dados da coluna JOB
df['JOB'].value_counts()

In [ ]:
# Transformar 'Other' em 0, 'ProfExe' em 1, 'Office'em 2, 'Mgr'em 3, 'Self'em 4, 'Sales' em 5
# nas coluna JOB
mapeamento = { 'Other': 0, 'ProfExe': 1, 'Office': 2, 'Mgr': 3, 'Self': 4, 'Sales': 5}

df_all['JOB'] = df_all['JOB'].replace(mapeamento).astype(int)

In [ ]:
#Verificar a transformação da coluna 'JOB'
df_all.head()

# Analise Exploratória

A Análise Exploratória de Dados consiste em analisar bases de dados e extrair informações úteis dos dados através de técnicas de visualizações.

In [ ]:
#Quantidade de registros por tipo de trabalho
df.groupby('JOB')['JOB'].count()

In [ ]:
#Descrever os dados quantitativos
df_all.describe()

In [ ]:
import seaborn as sns

sns.set_style('darkgrid')
sns.distplot(df['YOJ'])

# Modelo Random Forest

Random Forest é um algoritmo de aprendizagem de máquina flexível e fácil de usar que produz excelentes resultados a maioria das vezes, mesmo sem ajuste de hiperparâmetros.
* https://medium.com/machina-sapiens/o-algoritmo-da-floresta-aleat%C3%B3ria-3545f6babdf8

In [ ]:
# Definindo as colunas para treinamento
feats = [c for c in df_all.columns if c not in ['BAD']]

In [ ]:
#Apresentar as colunas
feats

In [ ]:
# importando a biblioteca para definir teste e treino

from sklearn.model_selection import train_test_split

In [ ]:
# Separar os dataframes
train, test = train_test_split(df_all, random_state=42)



In [ ]:
#Verificar a quantidade de registros de treino
print("Treino: ", train.shape)
print("Teste: ",test.shape)

In [ ]:
# Instanciando o random forest classifier utilizando 200 árvores
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=200)

In [ ]:
# Treinando o modelo
rf.fit(train[feats], train['BAD'])

In [ ]:
# Gerar as predicoes do modelo com os dados de teste
pred_test = rf.predict(test[feats])

In [ ]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
# Apresentar a acurácia do modelo
print("Accuracy:",metrics.accuracy_score(test['BAD'], pred_test))

accuracy = metrics.accuracy_score(test['BAD'], pred_test)

In [ ]:
# Apresentar a balanced do modelo
balanced = balanced_accuracy_score(test['BAD'],pred_test)
print("Balanced:",balanced)

In [ ]:
# Apresentar a F1 score do modelo
score = f1_score(test['BAD'],pred_test)
print("f1 score:",score)

In [ ]:
# Apresentar as colunas por importância
(pd.Series(rf.feature_importances_, index=feats)
   .nlargest(13)
   .plot(kind='barh')) 

In [ ]:
#Importar as bibliotecas
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
import scikitplot as skplt

In [ ]:
#Apresentar a matriz de confusão
matriz = confusion_matrix(test['BAD'], pred_test)

matriz

In [ ]:
# Apresentar o gráfico da matriz de confusão
skplt.metrics.plot_confusion_matrix(test['BAD'], pred_test)

# Conclusão

O dataset analisado tem 13 atributos e 3364 registros. 
Para a avaliação dos modelos, o dataset foi divido aleatoriamente em um conjunto para treinamento e um conjunto para teste. 
Foi criado 0 classificador floresta de árvores de decisão gerada pelo algoritmo Random Forest. 
A acurácia obtida foi de 95.4%.